In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
import warnings
from pathlib import Path
from glob import glob

import spikeinterface.full as si
import numpy as np 
import matplotlib.pyplot as plt
from pprint import pprint
from kneed import KneeLocator

import UnitMatchPy.overlord as ov
import UnitMatchPy.metric_functions as mf

sys.path.append("/home/phornauer/Git/spikesorting/")
from spikesorting import unit_match as um


warnings.filterwarnings("ignore", category=RuntimeWarning)

si.set_global_job_kwargs(n_jobs=16, progress_bar=False)

In [4]:
parent_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/'
project_name = 'Torsten_2'
recording_date = '241030'
chip_id = 'T002523'
assay_name = 'Network'
assay_id = '*'

well_ids = ["well{:03d}".format(w) for w in range(19,24)]
print(well_ids)

['well019', 'well020', 'well021', 'well022', 'well023']


In [5]:
for file_name in well_ids:

    path_parts = [parent_path, project_name, recording_date, chip_id, assay_name, assay_id, file_name, "sorter_output","spike_times.npy"]
    path_list = um.get_sorting_path_list(path_parts)
    waveforms, channel_pos = um.generate_templates(path_list)
    
#pprint(path_list)

Found 12 sorting paths matching the description:
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Torsten_2/241030/T002523/Network/*/well019/sorter_output/spike_times.npy

Extracting waveforms...


In [ ]:
recordings, sortings = um.load_objects(path_list)

In [ ]:
for i, sorting in enumerate(sortings):
    sortings[i] = um.select_good_units(sorting)
    
for recording in recordings:
    recording = si.highpass_filter(recording)

In [43]:
split_recordings, split_sortings = um.perform_cv_split(recordings, sortings)

In [44]:
analysers = []
for i, _ in enumerate(split_recordings):
    split_analysers = []

    split_analysers.append(
        si.create_sorting_analyzer(
            split_sortings[i][0], split_recordings[i][0], sparse=False
        )
    )
    split_analysers.append(
        si.create_sorting_analyzer(
            split_sortings[i][1], split_recordings[i][1], sparse=False
        )
    )
    analysers.append(split_analysers)

all_waveforms = []

In [ ]:
for i, _ in enumerate(analysers):
    for half in range(2):
        analysers[i][half].compute(
            [
                "random_spikes",
                "waveforms",
                "templates",
            ],
            extension_params={
                "random_spikes": {"max_spikes_per_unit": 500, "method": "uniform"}
                # "waveforms": {"ms_before": cutout_ms[0], "ms_after": cutout_ms[1]},
            },
        )
    templates_first = analysers[i][0].get_extension("templates")
    templates_second = analysers[i][1].get_extension("templates")
    t1 = templates_first.get_data()
    t2 = templates_second.get_data()

Exception ignored in: <function _releaseLock at 0x7fab5d2ce480>
Traceback (most recent call last):
  File "/home/phornauer/miniconda3/envs/si_env/envs/si101/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


In [30]:
split_sortings[1][1]

FrameSliceSorting: 6 units - 1 segments - 10.0kHz